In [163]:

import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.init as init
import torch.optim as optim
import random

In [164]:
width,height=4,4

In [165]:
import os,sys,inspect
sys.path.insert(1, os.path.join(sys.path[0], '..'))

In [166]:
%load_ext autoreload
%autoreload 2
import env.env as env
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [167]:
value_approximator=nn.Sequential(nn.Linear(18,4),nn.Linear(4,1))
states = [[i, j] for i in range(4) for j in range(4)]

In [168]:
state_tensor=torch.tensor(grid.current_state.flatten()[:]).float()

In [169]:
loss=nn.MSELoss()

In [176]:
optimizer = optim.SGD(value_approximator.parameters(), lr=0.001)

In [186]:
#One Episodestartx=random.randint(0,3)
grid=env.Grid_game(height,width)
Counter=np.zeros((width,height))
#starty=0
#startx=3
history=[]
Returns=np.zeros((height,width))
rewards=0
k=0.7

y,x=grid.player.curpos

num_episodes=1000


for i in range(num_episodes):
    state_list=[]
    starty,startx = random.choice(states[1:-1])
    grid.set_pos(starty,startx)
    iteration=0
    rewards=0
 #   if num_episodes % 1000:
      #  print(mse_loss)
    while(not grid.state):
        value_approximator.zero_grad()
        
        oldpos=grid.player.curpos[:]
      #  oldstate=torch.tensor(oldpos).float()
        oldstate=torch.cat([torch.tensor(grid.initial_board.flatten()).float(),torch.tensor(oldpos).float()])
        iteration+=1
        state,cur_state,reward,_=grid.transition()
        curpos=grid.player.curpos[:]
        newstate=torch.cat([torch.tensor(grid.initial_board.flatten()).float(),torch.tensor(curpos).float()])
        Counter[curpos[0]][curpos[1]]+=1
        
        val_newstate=-1+k*value_approximator(newstate)
        val_oldstate=value_approximator(oldstate)
        
        mse_loss=loss(val_oldstate,val_newstate)
        mse_loss.backward()
        optimizer.step()
        break
        
    history.append(grid.vst[:].copy())
    
    

In [187]:
for i in range(4):
    for j in range(4):
        grid.vst[i][j]=value_approximator(torch.cat([torch.tensor(grid.initial_board.flatten()).float(),torch.tensor([i,j]).float()]))

In [188]:
grid.vst

array([[-3.30407476, -3.32501984, -3.34596443, -3.3669095 ],
       [-3.3206892 , -3.34163427, -3.36257887, -3.38352394],
       [-3.33730364, -3.35824823, -3.37919331, -3.4001379 ],
       [-3.3539176 , -3.37486267, -3.39580774, -3.41675234]])

In [151]:
torch.cat([torch.tensor(grid.initial_board.flatten()).float(),torch.tensor([3,4]).float()])

tensor([-1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0., -1.,  3.,  4.])